In [638]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats import mode
from scipy.stats.distributions import randint, uniform, expon, reciprocal
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
import re

In [639]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

In [640]:
X_train = train.drop('Survived', axis=1)
y_train = train['Survived'].ravel()

In [641]:
def Get_Title(word):
    atr = re.search('(\w+)\.', word)
    if not atr:
        return 'No'
    return atr.group()

In [642]:
class Transform_Ticket(TransformerMixin, BaseEstimator):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        Xt['Ticket_Num'] = X['Ticket'].apply(lambda x: x.split(' ')[-1])
        return Xt

In [643]:
class Transform_Name_to_Title(TransformerMixin, BaseEstimator):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        Xt['Title'] = X['Name'].apply(Get_Title)
        return Xt

In [644]:
class Transform_Name_to_Surname(TransformerMixin, BaseEstimator):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        Xt['Surname'] = X['Name'].apply(lambda x: x.split(' ')[0])
        return Xt

In [658]:
class Surname_to_Feature(TransformerMixin, BaseEstimator):
    def __init__(self, high=0.7, low=0.3):
        self.high = high
        self.low = low
    def fit(self, X, y=None):
        Xt = Transform_Name_to_Surname().fit_transform(X)
        Xt['Survived'] = y.copy()
        dt = Xt.groupby('Surname')['Survived'].agg([('Count', np.size), ('Prop', np.mean)]).reset_index()
        self.SurHigh = list(dt[(dt['Count'] > 1) & (dt['Prop'] > self.high)]['Surname'])
        self.SurLow = list(dt[(dt['Count'] > 1) & (dt['Prop'] < self.low)]['Surname'])
        return self
    def transform(self, X, y=None):
        Xaux = Transform_Name_to_Surname().fit_transform(X).reset_index(drop=True)
        Xt = X.copy().reset_index(drop=True)
        Xt['High'] = 0
        Xt['Low'] = 0
        for i in range(len(X)):
            sur = Xaux.loc[i,'Surname']
            if sur in self.SurHigh:
                Xt.loc[i, 'High'] = 1
            elif sur in self.SurLow:
                Xt.loc[i, 'Low'] = 1
        return Xt        

In [646]:
def write_test(df, mod, n=1):
    dft = pd.DataFrame(np.c_[df['PassengerId'],mod.predict(df)], columns=['PassengerId', 'Survived'])
    dft.to_csv('Predictions_Teste{}.csv'.format(n), index=False)

In [647]:
class Impute_Better(BaseEstimator, TransformerMixin):
    """ Impute missing data in a more effective way, groupwise by Sex and Pclass"""
    def __init__(self, col='Age', func=np.mean, mode=2):
        self.col = col
        self.func = func
        self.mode = mode
    def fit(self, X, y=None):
        if self.mode == 2:
            self.vals = X.groupby(by=['Sex', 'Pclass'])[self.col].agg(self.func)
        elif self.mode == 1:
            self.vals = X.groupby(by=['Sex'])[self.col].agg(self.func)
        else:
            self.vals = X[self.col].agg(self.func)
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        idx_na = X.loc[:,self.col].isna()
        if self.mode == 2:
            for i in range(len(self.vals)):
                Xt.loc[((X['Sex'] == self.vals.index[i][0]) & (X['Pclass'] == self.vals.index[i][1]) &
                       idx_na),self.col] = self.vals[i]
        elif self.mode == 1:
            for i in range(len(self.vals)):
                Xt.loc[((X['Sex'] == self.vals.index[i]) & idx_na),self.col] = self.vals[i]
        else:
            Xt.loc[idx_na, self.col] = self.vals
        return Xt

In [648]:
class New_Features(BaseEstimator, TransformerMixin):
    """ Add new features, in this case Parch + SibSp """
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        Xt['Nrelatives'] = Xt['Parch'] + Xt['SibSp']
        return Xt

In [649]:
class Transf_Conti(BaseEstimator, TransformerMixin):
    """ Transforms a feature in categorical or startardizes it"""
    def __init__(self, n=4, q=False, conti=False):
        self.n = n
        self.q = q
        self.conti=conti
    def fit(self, X, y=None):
        if not self.conti:
            if not self.q:
                self.binst = pd.cut(X.values[:,0], bins=self.n, retbins=True, duplicates='drop')[1]
            if self.q:
                self.binst = pd.qcut(X.values[:,0], q=self.n, retbins=True, duplicates='drop')[1]
            self.higher = self.binst[-1]
            self.lower = self.binst[0]
        else:
            self.scaler_transf = StandardScaler()
            self.scaler_transf.fit(X)
        
        return self
    def transform(self, X, y=None):
        if not self.conti:
            val_ret = pd.cut(X.values[:,0], bins=self.binst, labels=False, include_lowest=True, duplicates='drop')
            val_ret[X.values[:,0] > self.higher] = self.n - 1
            val_ret[X.values[:,0] < self.lower] = 0
            return val_ret.reshape((-1, 1))
        else:
            return self.scaler_transf.transform(X)

In [650]:
class Select_Features(BaseEstimator, TransformerMixin):
    def __init__(self, select=False):
        self.select=select
    def fit(self, X, y=None):
        return self
    def transform(self, X,y=None):
        if not self.select:
            return X
        return X[:, self.select]

In [697]:
Pipe_Impute = Pipeline([
    #Impute Pipeline, the one which imputes missing data and creates new features
    ('Age_Impute', Impute_Better(col='Age', func=np.median)),
    ('Fare_Impute', Impute_Better(col='Fare', func=np.median)),
    #('Embarked_Impute', Impute_Better(col='Embarked', func=lambda x: mode(x)[0][0])),
    ('New_Features', New_Features()),
    ('Surname_to_Code', Surname_to_Feature())
])

In [698]:
Pipe_Preprocess = ColumnTransformer([
    ### Preprocessing pipeline. Encodes, standardizes
    ('Ordinal_Encode', OrdinalEncoder(), ['Sex','Pclass']), 
    ('Age_Encode', Transf_Conti(), ['Age']), 
    ('Fare_Encode', Transf_Conti(), ['Fare']), 
    ('Keep', 'passthrough', ['Parch', 'SibSp', 'Nrelatives', 'High', 'Low']), 
    #('Embarked_Encode', OneHotEncoder(), ['Embarked'])
])

In [709]:
FullPipePredict_RF = Pipeline([
    ('Impute_Pipeline', Pipe_Impute),
    ('Preprocess_Pipeline', Pipe_Preprocess),
    ('Feature_Selection', Select_Features()),
    ('Predict', RandomForestClassifier(n_jobs=-1))
])

In [723]:
param_RF = {
    'Impute_Pipeline__Surname_to_Code__high':[0.9],
    'Feature_Selection__select':[[0,1,2,3,6,7]],
    'Predict__n_estimators':randint(130,200),
    'Predict__max_leaf_nodes':randint(18,30),
}

In [724]:
grid_RF = RandomizedSearchCV(FullPipePredict_RF, param_distributions=param_RF, cv=10, n_iter=60,
                            scoring='accuracy', verbose=10)

In [725]:
grid_RF.fit(X_train, y_train)

Fitting 10 folds for each of 60 candidates, totalling 600 fits
[CV 1/10; 1/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=192
[CV 1/10; 1/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=192;, score=0.800 total time=   0.5s
[CV 2/10; 1/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=192
[CV 2/10; 1/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=192;, score=0.787 total time=   1.6s
[CV 3/10; 1/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=192
[CV 3/10; 1/60] END Feature_Selection__select

[CV 3/10; 3/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=27, Predict__n_estimators=194;, score=0.753 total time=   0.2s
[CV 4/10; 3/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=27, Predict__n_estimators=194
[CV 4/10; 3/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=27, Predict__n_estimators=194;, score=0.831 total time=   0.2s
[CV 5/10; 3/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=27, Predict__n_estimators=194
[CV 5/10; 3/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=27, Predict__n_estimators=194;, score=0.843 total time=   0.2s
[CV 6/10; 3/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pi

[CV 6/10; 5/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=28, Predict__n_estimators=137;, score=0.775 total time=   0.1s
[CV 7/10; 5/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=28, Predict__n_estimators=137
[CV 7/10; 5/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=28, Predict__n_estimators=137;, score=0.820 total time=   0.2s
[CV 8/10; 5/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=28, Predict__n_estimators=137
[CV 8/10; 5/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=28, Predict__n_estimators=137;, score=0.742 total time=   0.2s
[CV 9/10; 5/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pi

[CV 9/10; 7/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=23, Predict__n_estimators=147;, score=0.843 total time=   0.2s
[CV 10/10; 7/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=23, Predict__n_estimators=147
[CV 10/10; 7/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=23, Predict__n_estimators=147;, score=0.798 total time=   0.3s
[CV 1/10; 8/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=172
[CV 1/10; 8/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=172;, score=0.800 total time=   0.3s
[CV 2/10; 8/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_

[CV 2/10; 10/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=24, Predict__n_estimators=151;, score=0.787 total time=   0.2s
[CV 3/10; 10/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=24, Predict__n_estimators=151
[CV 3/10; 10/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=24, Predict__n_estimators=151;, score=0.753 total time=   0.2s
[CV 4/10; 10/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=24, Predict__n_estimators=151
[CV 4/10; 10/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=24, Predict__n_estimators=151;, score=0.831 total time=   0.2s
[CV 5/10; 10/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Imp

...

[CV 9/10; 53/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=27, Predict__n_estimators=171;, score=0.843 total time=   0.2s
[CV 10/10; 53/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=27, Predict__n_estimators=171
[CV 10/10; 53/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=27, Predict__n_estimators=171;, score=0.798 total time=   0.2s
[CV 1/10; 54/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=25, Predict__n_estimators=197
[CV 1/10; 54/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=25, Predict__n_estimators=197;, score=0.800 total time=   0.3s
[CV 2/10; 54/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], I

[CV 2/10; 56/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=18, Predict__n_estimators=178;, score=0.787 total time=   0.3s
[CV 3/10; 56/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=18, Predict__n_estimators=178
[CV 3/10; 56/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=18, Predict__n_estimators=178;, score=0.753 total time=   0.2s
[CV 4/10; 56/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=18, Predict__n_estimators=178
[CV 4/10; 56/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=18, Predict__n_estimators=178;, score=0.831 total time=   0.2s
[CV 5/10; 56/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Imp

[CV 5/10; 58/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=199;, score=0.843 total time=   0.3s
[CV 6/10; 58/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=199
[CV 6/10; 58/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=199;, score=0.775 total time=   0.2s
[CV 7/10; 58/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=199
[CV 7/10; 58/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=199;, score=0.820 total time=   0.2s
[CV 8/10; 58/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Imp

[CV 8/10; 60/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=197;, score=0.742 total time=   0.3s
[CV 9/10; 60/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=197
[CV 9/10; 60/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=197;, score=0.820 total time=   0.2s
[CV 10/10; 60/60] START Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=197
[CV 10/10; 60/60] END Feature_Selection__select=[0, 1, 2, 3, 6, 7], Impute_Pipeline__Surname_to_Code__high=0.9, Predict__max_leaf_nodes=29, Predict__n_estimators=197;, score=0.775 total time=   0.2s


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('Impute_Pipeline',
                                              Pipeline(steps=[('Age_Impute',
                                                               Impute_Better(func=<function median at 0x000001F499CA28B0>)),
                                                              ('Fare_Impute',
                                                               Impute_Better(col='Fare',
                                                                             func=<function median at 0x000001F499CA28B0>)),
                                                              ('New_Features',
                                                               New_Features()),
                                                              ('Surname_to_Code',
                                                               Surname_to_Feature())])),
                                             ('Preprocess_Pipeline',
                

In [728]:
grid_RF.best_score_

0.8024719101123596